# PyTorch Basics

Credit to https://github.com/yunjey for the original Python code found at: 

https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/pytorch_basics/main.py

In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

# Basic Autograd Example 1

Create tensors

In [2]:
x = torch.tensor(1., requires_grad = True)
w = torch.tensor(2., requires_grad = True)
b = torch.tensor(3., requires_grad = True)

Create computational graph

In [3]:
y = w * x + b
y

tensor(5., grad_fn=<AddBackward0>)

Compute and print gradients.

From documentation: `backward()` computes the sum of gradients of given tensors w.r.t. graph leaves.

In [4]:
y.backward()

In [5]:
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


# Basic Autograd Example 2

Create tensors of particular shapes.

In [6]:
x = torch.randn(10, 3)
y = torch.randn(10, 2)

Apply a linear transformation to the incoming data

In [7]:
linear = nn.Linear(3, 2)
print("w:", linear.weight, '\n')
print("b:", linear.bias)

w: Parameter containing:
tensor([[ 0.1713, -0.0831,  0.0481],
        [-0.1429, -0.2109, -0.3604]], requires_grad=True) 

b: Parameter containing:
tensor([-0.5484, -0.1210], requires_grad=True)


Assign loss function and optimizer.

In [8]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.1)

Make a prediction

In [9]:
preds = linear(x)

Compute loss

In [10]:
loss = criterion(preds, y)
print("Loss:", loss.item())

Loss: 1.139244556427002


Backwards pass

In [11]:
loss.backward()

Print out the gradients

In [20]:
print("dL/dW:", linear.weight.grad, "\n")
print("dL/db:", linear.bias.grad)

dL/dW: tensor([[-0.0767, -0.3645,  0.3215],
        [-0.0696, -0.2950, -0.3836]]) 

dL/db: tensor([-0.3688, -0.0233])


One step gradient descent

In [19]:
optimizer.step()

Print out the loss after one step of optimization

In [21]:
pred = linear(x)
loss = criterion(preds, y)
print("Loss after 1 step of optimizaton:", loss.item())

Loss after 1 step of optimizaton: 1.139244556427002


# Loading Data from a numpy array

Create numpy array.

In [30]:
x = np.array([[1, 2], [3, 4]], dtype=int)
print(x)

[[1 2]
 [3 4]]


Convert array to torch tensor.

`.from_numpy()` creates a tensor that shares the same memory as the source nd.array

Thus, if the numpy array changes, so does the tensor.

In [99]:
y1 = torch.tensor(x)
y2 = torch.from_numpy(x)

print("torch.tensor():", y1)
print("torch.from_numpy():", y2)
print("torch.from_numpy():", y3)


x[0][0] = 2

print("\nAfter changing x[0][0] to 2:")
print("torch.tensor():", y1[0][0])
print("torch.from_numpy():", y2[0][0])

x[0][0] = 1

torch.tensor(): tensor([[1, 2],
        [3, 4]])
torch.from_numpy(): tensor([[1, 2],
        [3, 4]])
torch.from_numpy(): ToTensor()

After changing x[0][0] to 2:
torch.tensor(): tensor(1)
torch.from_numpy(): tensor(2)


Convert torch tensor to a numpy array.

In [43]:
y2 = y2.numpy()
print(y2)

[[1 2]
 [3 4]]


# Input Pipeline for dataset that's part of PyTorch

All `datasets` are subclasses of torch.utils.data.Dataset i.e, they have `__getitem__` and `__len__` methods implemented

In [54]:
training_dataset = torchvision.datasets.CIFAR10(root='data',
                                                train=True,
                                                transform=transforms.ToTensor(),
                                                download=True)

Files already downloaded and verified


In [73]:
len(training_dataset)

50000

Fetch one data pair

In [59]:
image, label = training_dataset[0]
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


You can index size by passing in the index value. For example, if you just want the first object in the size list (in this case, the number of channels)

In [101]:
image.size(0)

3

Data loader for packaging the data into batches and eventually grabbing batches for training.

In [60]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=64,
                                          shuffle=True)

 When iteration starts, queue and thread start to load data from files.

In [61]:
data_iterator = iter(train_loader)

Mini-batch of images and their labels.

In [65]:
images, labels = next(data_iterator)
print(images.size())
print(labels)

torch.Size([64, 3, 32, 32])
tensor([7, 7, 3, 5, 3, 6, 5, 4, 9, 7, 3, 6, 2, 6, 9, 2, 6, 2, 3, 8, 3, 9, 7, 0,
        5, 5, 7, 0, 0, 9, 1, 7, 9, 8, 6, 6, 0, 4, 1, 4, 2, 3, 1, 8, 5, 3, 9, 4,
        0, 9, 5, 5, 1, 2, 9, 7, 6, 2, 6, 9, 9, 0, 9, 3])


Actual usage of the data loader:

In [63]:
for images, labels in train_loader:
    # Training code
    pass

# Input pipeline for a custom dataset

Framework for a custom dataset

In [94]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # Initialize file paths or list of file names
        pass
    def __getitem__(self, index):
        # 1. Read one data from file, e.g. numpy.fromfile or PIL.Image.open
        # 2. Preprocess the data, e.g. torch.transform()
        # 3. Return a data pair, e.g. image and label
        pass
    def __len__(self):
        # Should return the length of the dataset
        # 128 is an arbitrary number just to make later steps work
        return 128

After creating the `Dataset`, you can use the prebuilt `DataLoader`

In [80]:
custom_dataset = CustomDataset()
custom_training_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                                    batch_size=64,
                                                    shuffle=True)

# Using a Pretrained Model

Example with resnet18

In [81]:
resnet = torchvision.models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/jamesteow/.torch/models/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:02<00:00, 17176361.04it/s]


Fine-tuning the top layer.

In [85]:
for param in resnet.parameters():
    param.requires_grad = False

Replace the classification layer (top layer)

In [86]:
resnet.fc = nn.Linear(resnet.fc.in_features, 100)

Linear(in_features=512, out_features=1000, bias=True)

Forward pass

In [87]:
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print(outputs.size())

torch.Size([64, 1000])


# Saving and Loading Models

In [88]:
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

Saving and loading only the model parameters

In [93]:
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))